## Exploratory Time Series Analysis with Reddit Based Lucey Uncertainty Index
* Identifying if our baseline ```Reddit UCRY``` indices serve as a leading indicator for various cryptocurrencies
* Further analysis on its relationship with existing volatility and returns indicators for crypto
* Locate any relationships with other asset classes (E.g. SP500, Gold, etc.)

### Set Up

In [92]:
# NB config
%load_ext autoreload
%autoreload 2

# Load Libraries
import os
import toml
import warnings
import numpy as np
import scipy.stats as stats
import altair as alt
os.chdir("..")
from pathlib import Path
import pandas as pd
from darts import TimeSeries
from darts.utils import statistics 
from sqlalchemy import create_engine

# Suppress warnings
warnings.filterwarnings("ignore")

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
# Connect to postgres
pg_config = (
    toml.load(Path() / "config" / "etl_config.toml")
    ["postgres"]

)
pg_engine = (
    create_engine(pg_config["default_local_uri"],
                  echo=True)
)

### Pull Relevant Data

In [3]:
# Set Date Range
START_DATE, END_DATE = "2014-01-01", "2021-12-31"

In [4]:
# Original Lucey UCRY Index Data
index_data_path = Path("/Users/christopherliew/Desktop/Y4S1/HT/crypto_uncertainty_index/pipelines/crypto_index") / "index_data"

ucry_original = pd.read_csv(index_data_path / "ucry_lucey_original.csv")
ucry_original_policy = ucry_original[["Timeline", "UCRY Policy Index"]]
ucry_original_price = ucry_original[["Timeline", "UCRY Price Index"]]

# Process dates
# Break up timeline column
ucry_original_policy[["start_date", "end_date"]] = (
    ucry_original_policy["Timeline"]
    .str.split(" - ", expand=True)
)

ucry_original_policy.rename(columns={"UCRY Policy Index": "index_value"}, inplace=True)
ucry_original_policy.drop(columns=["Timeline"], inplace=True)
ucry_original_policy["type"] = "Lucey-Original-Policy"

ucry_original_price[["start_date", "end_date"]] = (
    ucry_original_price["Timeline"]
    .str.split(" - ", expand=True)
).rename(columns={"UCRY Price Index": "index_value"})

ucry_original_price.rename(columns={"UCRY Price Index": "index_value"}, inplace=True)
ucry_original_price.drop(columns=["Timeline"], inplace=True)
ucry_original_price["type"] = "Lucey-Original-Price"


In [5]:
# UCRY Index Data
# Lucey Price
ucry_query = f"""
    SELECT *
    FROM ucry_index ui
    """

ucry_reddit = pd.read_sql(sql=ucry_query, con=pg_engine)
ucry_reddit["type"] = ucry_reddit["type"].map({"lucey-price": "Lucey-Reddit-Price", "lucey-policy": "Lucey-Reddit-Policy"})

ucry_price = ucry_reddit[ucry_reddit.type == "Lucey-Reddit-Price"]
ucry_policy = ucry_reddit[ucry_reddit.type == "Lucey-Reddit-Policy"]

2022-02-11 03:39:32,170 INFO sqlalchemy.engine.Engine select pg_catalog.version()
2022-02-11 03:39:32,171 INFO sqlalchemy.engine.Engine [raw sql] {}
2022-02-11 03:39:32,175 INFO sqlalchemy.engine.Engine select current_schema()
2022-02-11 03:39:32,175 INFO sqlalchemy.engine.Engine [raw sql] {}
2022-02-11 03:39:32,178 INFO sqlalchemy.engine.Engine show standard_conforming_strings
2022-02-11 03:39:32,178 INFO sqlalchemy.engine.Engine [raw sql] {}
2022-02-11 03:39:32,182 INFO sqlalchemy.engine.Engine select relname from pg_class c join pg_namespace n on n.oid=c.relnamespace where pg_catalog.pg_table_is_visible(c.oid) and relname=%(name)s
2022-02-11 03:39:32,182 INFO sqlalchemy.engine.Engine [generated in 0.00041s] {'name': '\n    SELECT *\n    FROM ucry_index ui\n    '}
2022-02-11 03:39:32,188 INFO sqlalchemy.engine.Engine 
    SELECT *
    FROM ucry_index ui
    
2022-02-11 03:39:32,189 INFO sqlalchemy.engine.Engine [raw sql] {}


In [6]:
# Combined UCRY
ucry_combined = (
    pd.concat([ucry_reddit, ucry_original_price, ucry_original_policy], axis=0)
)
ucry_combined["start_date"] = pd.to_datetime(ucry_combined["start_date"], utc=True)
ucry_combined["end_date"] = pd.to_datetime(ucry_combined["end_date"], utc=True)
ucry_combined.head()

,start_date,end_date,doc_count,index_value,type
0,2013-12-30 00:00:00+00:00,2014-01-05 23:59:59.999999+00:00,1.0,99.160104,Lucey-Reddit-Price
1,2014-01-06 00:00:00+00:00,2014-01-12 23:59:59.999999+00:00,5.0,99.426172,Lucey-Reddit-Price
2,2014-01-13 00:00:00+00:00,2014-01-19 23:59:59.999999+00:00,9.0,99.692240,Lucey-Reddit-Price
3,2014-01-20 00:00:00+00:00,2014-01-26 23:59:59.999999+00:00,6.0,99.492689,Lucey-Reddit-Price
4,2014-01-27 00:00:00+00:00,2014-02-02 23:59:59.999999+00:00,4.0,99.359655,Lucey-Reddit-Price


In [7]:
ucry_combined["type"].unique()

array(['Lucey-Reddit-Price', 'Lucey-Reddit-Policy',
       'Lucey-Original-Price', 'Lucey-Original-Policy'], dtype=object)

In [8]:
# Crypto Price Data
asset_types_query = """
    SELECT DISTINCT ticker
    FROM asset_prices
    """

asset_types = [i for i in pg_engine.execute(asset_types_query)]

2022-02-11 03:39:32,355 INFO sqlalchemy.engine.Engine 
    SELECT DISTINCT ticker
    FROM asset_prices
    
2022-02-11 03:39:32,356 INFO sqlalchemy.engine.Engine [raw sql] {}


In [9]:
# Get respective time series
combined_df = pd.read_sql(sql="SELECT * FROM asset_prices", con=pg_engine)
btc_usd = combined_df[combined_df.ticker == "BTC-USD"]
eth_usd = combined_df[combined_df.ticker == "ETH-USD"]
luna_usd = combined_df[combined_df.ticker == "LUNA-USD"]
tether_usd = combined_df[combined_df.ticker == "USDT-USD"]
ripple_usd = combined_df[combined_df.ticker == "XRP-USD"]
polkadot_usd = combined_df[combined_df.ticker == "DOT-USD"]
cardano_usd = combined_df[combined_df.ticker == "ADA-USD"]
binance_usd = combined_df[combined_df.ticker == "BNB-USD"]

2022-02-11 03:39:32,407 INFO sqlalchemy.engine.Engine select relname from pg_class c join pg_namespace n on n.oid=c.relnamespace where pg_catalog.pg_table_is_visible(c.oid) and relname=%(name)s
2022-02-11 03:39:32,408 INFO sqlalchemy.engine.Engine [cached since 0.2262s ago] {'name': 'SELECT * FROM asset_prices'}
2022-02-11 03:39:32,419 INFO sqlalchemy.engine.Engine SELECT * FROM asset_prices
2022-02-11 03:39:32,419 INFO sqlalchemy.engine.Engine [raw sql] {}


In [10]:
# Other Data
sp500 = combined_df[combined_df.ticker == "^GSPC"]
gold = combined_df[combined_df.ticker == "GC=F"]

### Exploratory Analysis: Identifying if UCRY Lucey (Reddit Based) is a Leading Indicator
#### Overview of Approach
1. Convert to Darts' TimeSeries
2. Price and Volume at different Horizons
   * Adjusted Closing Price hereafter ```Price```
   * Trade Volumes 
3. Returns and Volatility at different Horizons
   * Price Returns
   * Price Volatility


#### 1. Convert to Darts TimeSeries

In [11]:
# Helper to convert to TS Dataset
def pd_to_darts(df: pd.DataFrame, time_col: str = "date") -> TimeSeries:
    new_df = TimeSeries.from_dataframe(
        df.reset_index(drop=True)
        .drop(columns=["ticker"]),
        time_col=time_col
    )
    return new_df

In [12]:
# Convert to Time Series Datasets
btc_usd_ts = pd_to_darts(btc_usd)
eth_usd_ts = pd_to_darts(eth_usd)
luna_usd_ts = pd_to_darts(luna_usd)
tether_usd_ts = pd_to_darts(tether_usd)
ripple_usd_ts = pd_to_darts(ripple_usd)
polkadot_usd_ts = pd_to_darts(polkadot_usd)
cardano_usd_ts = pd_to_darts(cardano_usd)
binance_usd_ts = pd_to_darts(binance_usd)
sp500_ts = pd_to_darts(sp500)
gold_ts = pd_to_darts(gold)

#### 2 Price and Volume Analyses
##### 2.1 UCRY against Asset Prices Analysis
Plot UCRY price and policy against Prices = Adjusted Closing Price (Weekly Granularity)
1. UCRY vs Prices at ***h = 0 weeks***
2. UCRY vs Prices at ***h = 4 weeks*** 
3. UCRY vs Prices at ***h = 12 weeks***
4. UCRY vs Prices at ***h = 24 weeks***

```Add in Correlation Coefficient to Charts```

**Dataset Prep**

In [74]:
# Prep Datasets
asset = (
    combined_df[["date", "ticker", "adj_close"]]
    .rename(columns={"date": "Date", "ticker": "Ticker", "adj_close": "Price"})
)

# Select relevant UCRY indices
ucry = (
    ucry_combined[["start_date", "type", "index_value"]]
    .rename(columns={"start_date": "Date", "type": "Index", "index_value": "Index Value (Standardised)"})
    [ucry_combined["type"].isin(["Lucey-Reddit-Price", "Lucey-Reddit-Policy"])]
)

In [75]:
# Generate lags
lags_week = [4, 12, 24, 52]

for lag in lags_week:
    asset[f'Price (Lag={lag})'] = asset['Price'].shift(-lag)

asset.head()

,Date,Ticker,Price,Price (Lag=4),Price (Lag=12),Price (Lag=24),Price (Lag=52)
0,2014-09-15,BTC-USD,398.821014,389.545990,351.631989,274.354004,231.212006
1,2014-09-22,BTC-USD,377.181000,354.704010,320.842987,286.393005,232.757004
2,2014-09-29,BTC-USD,320.510010,325.891998,317.239014,267.959991,238.259003
3,2014-10-06,BTC-USD,378.549011,363.264008,264.195007,242.712997,247.050003
4,2014-10-13,BTC-USD,389.545990,387.881989,265.660004,260.597992,261.643005


**Helper to Generate Faceted Chart of Price against UCRY**

In [98]:
def gen_price_vs_ucry_chart(lags_in_weeks=None):

    # Select horizon
    y_label = f"Price (Lag={lags_in_weeks})" if lags_in_weeks is not None else "Price"
    data = asset[["Date", "Ticker", y_label]]

    # Compute correlation coefficient
    # r, p = stats.pearsonr(
    #    data.dropna()[y_label],
    #    ucry.dropna()["Index Value (Standardised)"],
    #)

    # Asset Class Line Chart
    asset_line = alt.Chart().mark_line(interpolate='basis').encode(
        x='Date:T',
        y=alt.Y(f'{y_label}:Q', scale=alt.Scale(type="log"), title=f'Lagged Price (Log-Scale) by {0 if lags_in_weeks is None else lags_in_weeks} Weeks'),
        # color='Ticker:N',
        color=alt.value("#FF0000"),
        tooltip=['Date', y_label, 'Ticker']
    )

    # UCRY Lucey Line Chart
    ucry_line = alt.Chart(ucry).mark_line(interpolate='basis').encode(
        x='Date:T',
        y=alt.Y('Index Value (Standardised):Q', scale=alt.Scale(domain=[93, 108])),
        color=alt.Color('Index:N', scale=alt.Scale(scheme='lighttealblue')),
        strokeDash='Index:N',
        tooltip=['Date', 'Index Value (Standardised)', 'Index']
    )

    # Generate multiple charts for each Ticker
    price_chart_gen = (
        alt.layer(asset_line, ucry_line, data=data.dropna(), title=f"{asset_name} Price vs UCRY Indices" )
        .transform_filter(alt.datum.Ticker == asset_name)
        .resolve_scale(y="independent", x="independent")
        for asset_name in asset.Ticker.unique()
    )

    alt.concat(*price_chart_gen, columns=4).display()

**Prices where h = 0 weeks**

In [99]:
gen_price_vs_ucry_chart()

alt.ConcatChart(...)

**Price where h = 4 Weeks**

In [100]:
gen_price_vs_ucry_chart(4)

alt.ConcatChart(...)

**Price where h = 12 Weeks**

In [101]:
gen_price_vs_ucry_chart(12)

alt.ConcatChart(...)

**Price where h = 24 Weeks**

In [103]:
gen_price_vs_ucry_chart(24)

alt.ConcatChart(...)

##### 2.2 UCRY against Asset Volume Analysis
Plot UCRY price and policy against Prices = Trading Volume (Weekly Granularity)
1. UCRY vs Volume at ***h = 0***
2. UCRY vs Volume at ***h = 1*** 
3. UCRY vs Volume at ***h = 3***
4. UCRY vs Volume at ***h = 6***
5. UCRY vs Volume at ***h = 12***

In [15]:
# Create Dual Axis with Index and Asset Volume at Different Horizons


#### 3 Price Returns and Volatility Analyses
##### 3.1 Create Price Return and Volatility Variables for various Horizons

##### 3.2 UCRY against Asset Price Returns
Plot UCRY price and policy against Price Returns
1. UCRY vs Prices Returns at ***h = 0***
2. UCRY vs Prices Returns at ***h = 1*** 
3. UCRY vs Prices Returns at ***h = 3***
4. UCRY vs Prices Returns at ***h = 6***
5. UCRY vs Prices Returns at ***h = 12***

In [16]:
# Compute Price Returns at Different Horizons

##### 3.3 UCRY against Asset Volatility
Plot UCRY price and policy against Volatility
1. UCRY vs Volatility at ***h = 0***
2. UCRY vs Volatility at ***h = 1*** 
3. UCRY vs Volatility at ***h = 3***
4. UCRY vs Volatility at ***h = 6***
5. UCRY vs Volatility at ***h = 12***

In [17]:
# Compute Volatility at Different Horizons

### Further Correlation Analysis
1. TS Preprocessing (Find Stationarize, autocorrelations, detrend and remove seasonality)
2. Cross Correlations